In [1]:
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import torch
import time
import soundfile as sf

Flash attention 2 is not installed


## Efficient Attention

In [2]:
prompts = [
    "Hello world.",
    "A futuristic AI assistant responding in a clear, robotic yet friendly manner",
    "A dramatic voice-over for an action-packed movie trailer that builds suspense",
    "A casual and friendly conversation starter that feels natural and engaging",
    "An enchanting introduction to a bedtime story that sparks imagination",
    "A detailed weather update that provides temperature, wind speed, and overall forecast",
    "A robotic voice delivering a monotone yet precise system status update",
    "A powerful and energetic motivational speech that inspires action and confidence"
]

descriptions = [
    "A female speaker delivers a warm and welcoming message with a slightly expressive and friendly tone. The speech has a moderate pace and a natural intonation, making it feel inviting. The recording is clear, with minimal background noise.",
    "A robotic AI voice speaks in a neutral yet slightly friendly manner. The speech is steady, with a consistent pitch and minimal variation in tone. The audio is crisp, resembling a synthesized assistant's response.",
    "A deep male voice narrates with a dramatic and intense tone, building suspense with pauses and rising intensity. The speech is slow-paced, emphasizing key moments. The recording is cinematic, with a slight reverberation for a grand effect.",
    "A young adult male speaker talks in a casual and relaxed tone. The speech is natural, with slight variations in pitch and pauses that mimic real-life conversations. The recording is high quality, making it feel like a personal chat.",
    "A soft-spoken female speaker introduces a bedtime story with a soothing and melodic tone. The pace is slow and gentle, making it easy to follow. The recording has a slight warmth, resembling a close-up microphone capture.",
    "A professional male voice delivers a clear and informative weather update. The tone is neutral but engaging, with a moderate pace. The articulation is precise, and the recording is high quality, with no distortions.",
    "A synthetic robotic voice speaks in a completely monotone and even-paced manner. The pitch remains constant, with no emotional inflection. The recording is crisp and clean, resembling an automated system response.",
    "A dynamic and energetic male speaker delivers an inspiring speech with a strong, enthusiastic tone. The pace is varied, with emphasis on key words to motivate the listener. The recording is sharp and immersive, with no background noise."
]

In [3]:
torch_device = "cuda:0" 
torch_dtype = torch.bfloat16
model_name = "parler-tts/parler_tts_mini_v0.1"

Using SDPA Attention

In [4]:
max_length = 70


tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = ParlerTTSForConditionalGeneration.from_pretrained(model_name,attn_implementation="sdpa").to(torch_device, dtype=torch_dtype)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/system4/anaconda3/envs/tf-gpu-211/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
 

In [5]:
input_ids = tokenizer(descriptions, return_tensors="pt",padding="max_length",max_length=70).to(torch_device)
prompt_input_ids = tokenizer(prompts, return_tensors="pt",padding="max_length",max_length=70).to(torch_device)

In [22]:
latencies = []
with torch.no_grad():
    for i in range(5):
        start_time = time.time()
        generation = model.generate(input_ids=input_ids.input_ids,
                    attention_mask=input_ids.attention_mask,
                    prompt_input_ids=prompt_input_ids.input_ids,
                    prompt_attention_mask=prompt_input_ids.attention_mask)
        end_time = time.time()
        pytorch_latency = end_time - start_time
        latencies.append(pytorch_latency)
        print(f"⏱️ PyTorch Inference Time Run {i+1}: {pytorch_latency:.4f} seconds")
    
    avg_latency = sum(latencies) / len(latencies)
    print(f"\n⏱️ Average PyTorch Inference Time: {avg_latency:.4f} seconds")

# audio_arr = generation.cpu().numpy().squeeze()
# sf.write(output_file, audio_arr, model.config.sampling_rate)

⏱️ PyTorch Inference Time Run 1: 14.5462 seconds
⏱️ PyTorch Inference Time Run 2: 16.7351 seconds
⏱️ PyTorch Inference Time Run 3: 16.6038 seconds
⏱️ PyTorch Inference Time Run 4: 16.9492 seconds
⏱️ PyTorch Inference Time Run 5: 19.7515 seconds

⏱️ Average PyTorch Inference Time: 16.9172 seconds


: 

In [ ]:
output_file = "generated_audio.wav"
audio_arr = generation[1].cpu().to(torch.float32).numpy().squeeze()
sf.write(output_file, audio_arr, model.config.sampling_rate)

Using Normal Default/Eager Attention

In [ ]:
max_length = 70

tokenizer = AutoTokenizer.from_pretrained(model_name) 
new_model = ParlerTTSForConditionalGeneration.from_pretrained(
    model_name, attn_implementation="eager").to(torch_device)

Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_si

In [ ]:
latencies = []
with torch.no_grad():
    for i in range(5):
        start_time = time.time()
        generation = new_model.generate(input_ids=input_ids.input_ids,
                    attention_mask=input_ids.attention_mask,
                    prompt_input_ids=prompt_input_ids.input_ids,
                    prompt_attention_mask=prompt_input_ids.attention_mask)
        end_time = time.time()
        pytorch_latency = end_time - start_time
        latencies.append(pytorch_latency)
        print(f"⏱️ PyTorch Inference Time Run {i+1}: {pytorch_latency:.4f} seconds")
    
    avg_latency = sum(latencies) / len(latencies)
    print(f"\n⏱️ Average PyTorch Inference Time: {avg_latency:.4f} seconds")

# audio_arr = generation.cpu().numpy().squeeze()
# sf.write(output_file, audio_arr, model.config.sampling_rate)

⏱️ PyTorch Inference Time Run 1: 21.4773 seconds
⏱️ PyTorch Inference Time Run 2: 19.0116 seconds
⏱️ PyTorch Inference Time Run 3: 22.9738 seconds
⏱️ PyTorch Inference Time Run 4: 15.0875 seconds
⏱️ PyTorch Inference Time Run 5: 18.5706 seconds

⏱️ Average PyTorch Inference Time: 19.4242 seconds


In [ ]:
output_file = "generated_audio.wav"
audio_arr = generation[1].cpu().to(torch.float32).numpy().squeeze()
sf.write(output_file, audio_arr, model.config.sampling_rate)